In [1]:
# Load packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

from FDApy import DenseFunctionalData, MultivariateFunctionalData
from FDApy.representation import DenseArgvals, DenseValues
from FDApy.preprocessing import MFPCA
from FDApy.visualization import plot, plot_multivariate

In [2]:
# Load shots
with open('./data/players_shots_density.pickle', 'rb') as f:
    players_shots_density = pickle.load(f)
with open('./data/players_shots_density_made.pickle', 'rb') as f:
    players_shots_density_made = pickle.load(f)

In [3]:
# Sort by ID
players_shots_density = players_shots_density.sort_values(by='PLAYER_ID')
players_shots_density_made = players_shots_density_made.sort_values(by='PLAYER_ID')

In [4]:
# Create functional data
argvals = DenseArgvals({
    'input_dim_0': np.linspace(0, 1, 51),
    'input_dim_1': np.linspace(0, 1, 51)
})

values_shots = DenseValues(
    np.stack(players_shots_density['DENSITY'].to_numpy())
)

values_made = DenseValues(
    np.stack(players_shots_density_made['DENSITY'].to_numpy())
)


fdata_shots = DenseFunctionalData(argvals, values_shots)
fdata_made = DenseFunctionalData(argvals, values_made)
fdata = MultivariateFunctionalData([fdata_shots, fdata_made])

In [5]:
# MFPCA
univariate_expansions = [
    {'method': 'FCPTPA', 'n_components': 30},
    {'method': 'FCPTPA', 'n_components': 30}
]
mfpca = MFPCA(n_components=20, method='covariance', univariate_expansions=univariate_expansions)
mfpca.fit(fdata)

/Users/steven/.virtualenvs/test_fdapy_venv/lib/python3.9/site-packages/FDApy/representation/functional_data.py:1119: UserWarning: The estimation of the variance of the noise is not performed for data with dimension larger than 1 and is set to 0.
  warnings.warn(


In [6]:
# Compute the scores
scores = mfpca.transform()

In [7]:
# Inverse transform
fdata_recons = mfpca.inverse_transform(scores)

In [8]:
# Save MFPCA results
with open('./data/MFPCA_FCPTPA.pickle', 'wb') as f:
    pickle.dump(mfpca, f, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
# Save scores
with open('./data/scores_FCPTPA.pickle', 'wb') as f:
    pickle.dump(scores, f, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
# Save reconstruction
with open('./data/MFPCA_FCPTPA_reconstruction.pickle', 'wb') as f:
    pickle.dump(fdata_recons, f, protocol=pickle.HIGHEST_PROTOCOL)